1. Pandas profiler
2. Check for outliers
3. Metric of interest correlation matrix, visualization by group (bar), visualization by age (count plot or line)
4. Create label for new users
5. If classification:
    -Random forest
    -LIME
    -Classification report
    -Confusion matrix
    -AUC

In [1]:
import lime
import numpy as np
import pandas as pd
import pandas_profiling
import sklearn
import sklearn.ensemble
import sklearn.metrics
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from sklearn import cluster
from sklearn.metrics import davies_bouldin_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler

/Users/johannaanderson/anaconda/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
def unique(list):
    x = np.array(list)
    return np.unique(x)
    

In [3]:
sdata = pd.read_csv('featuresdf.csv')
sdata.shape

(100, 16)

In [4]:
numsongs = sdata.groupby('artists')[['id']].nunique()
numsongs.head()

,id
artists,
21 Savage,1
AJR,1
Alessia Cara,1
Alok,1
Ariana Grande,1


In [5]:
numtrx = pd.merge(sdata, numsongs, on=['artists'])
numtrx.head(10)

,id_x,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,id_y
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1.0,-3.183,0.0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,233713.0,4.0,4
1,6PCUP3dWmTjcTtXY02oFd,Castle on the Hill,Ed Sheeran,0.461,0.834,2.0,-4.868,1.0,0.0989,0.0232,0.000011,0.1400,0.471,135.007,261154.0,4.0,4
2,0afhq8XCExXpqazXczTSv,Galway Girl,Ed Sheeran,0.624,0.876,9.0,-3.374,1.0,0.1000,0.0735,0.000000,0.3270,0.781,99.943,170827.0,4.0,4
3,0tgVpDi06FyKpA1z0VMD4,Perfect,Ed Sheeran,0.599,0.448,8.0,-6.312,1.0,0.0232,0.1630,0.000000,0.1060,0.168,95.050,263400.0,3.0,4
4,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2.0,-4.328,1.0,0.1200,0.2290,0.000000,0.0924,0.813,88.931,228827.0,4.0,2
5,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.846,177.833,228200.0,4.0,2
6,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11.0,-6.769,0.0,0.0317,0.0498,0.000014,0.1640,0.446,103.019,247160.0,4.0,4
7,72jbDTw1piOOj770jWNea,Paris,The Chainsmokers,0.653,0.658,2.0,-6.428,1.0,0.0304,0.0215,0.000002,0.0939,0.219,99.990,221507.0,4.0,4
8,7BKLCZ1jbUBVqRi2FVlTV,Closer,The Chainsmokers,0.748,0.524,8.0,-5.599,1.0,0.0338,0.4140,0.000000,0.1110,0.661,95.010,244960.0,4.0,4
9,0QsvXIfqM0zZoerQfsI9l,Don't Let Me Down,The Chainsmokers,0.542,0.859,11.0,-5.651,1.0,0.1970,0.1600,0.004660,0.1370,0.403,159.797,208053.0,4.0,4


In [9]:
artmeans = numtrx.groupby('artists')['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms'].mean()

In [12]:
artmeans.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
artists,,,,,,,,,,
21 Savage,0.884,0.346,-8.228,0.3510,0.0151,0.000007,0.0871,0.376,75.016,220307.0
AJR,0.673,0.637,-4.518,0.0429,0.1370,0.000000,0.1840,0.678,123.980,201160.0
Alessia Cara,0.314,0.555,-9.601,0.3700,0.1570,0.000108,0.0670,0.159,179.666,175517.0
Alok,0.789,0.442,-7.844,0.0421,0.5860,0.003660,0.0927,0.450,121.971,192846.0
Ariana Grande,0.648,0.738,-5.883,0.2470,0.0408,0.000000,0.2920,0.603,159.145,226160.0


In [18]:
artmodes = numtrx.groupby('artists')['key', 'mode', 'time_signature'].agg(pd.Series.mode)
artmodes.head()

,key,mode,time_signature
artists,,,
21 Savage,8,0,4.0
AJR,5,1,4.0
Alessia Cara,9,1,4.0
Alok,11,1,4.0
Ariana Grande,6,0,4.0


In [25]:
artists = artmeans.join(artmodes)
artistsfinal = pd.merge(artists, numtrx[['artists', 'id_y']], on='artists')
artistsfinal = artistsfinal.rename(columns={"id_y":"numtrx"})

In [26]:
artistsfinal.head()

,artists,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,key,mode,time_signature,numtrx
0,21 Savage,0.884,0.346,-8.228,0.3510,0.0151,0.000007,0.0871,0.376,75.016,220307.0,8,0,4.0,1
1,AJR,0.673,0.637,-4.518,0.0429,0.1370,0.000000,0.1840,0.678,123.980,201160.0,5,1,4.0,1
2,Alessia Cara,0.314,0.555,-9.601,0.3700,0.1570,0.000108,0.0670,0.159,179.666,175517.0,9,1,4.0,1
3,Alok,0.789,0.442,-7.844,0.0421,0.5860,0.003660,0.0927,0.450,121.971,192846.0,11,1,4.0,1
4,Ariana Grande,0.648,0.738,-5.883,0.2470,0.0408,0.000000,0.2920,0.603,159.145,226160.0,6,0,4.0,1


In [39]:
# Hierarchical clustering for unlabeled data
clus_data = StandardScaler().fit_transform(artmeans)
# connectivity matrix for structured Ward
connectivity = kneighbors_graph(clus_data, n_neighbors=10, include_self=False)
# make connectivity symmetric
connectivity = 0.5 * (connectivity + connectivity.T)

ward = cluster.AgglomerativeClustering(n_clusters=2, linkage='ward', connectivity=connectivity)

In [40]:
clust_mod = ward.fit(artmeans)
labels = clust_mod.labels_
label_df = pd.DataFrame(labels, columns=['Labels'])
label_df.shape

(78, 1)

In [41]:
# Davies-Bouldin 5 clust: 0.96
#                4 clust: 0.55
#                3 clust: 0.45
#                2 clust: 0.19

davies_bouldin_score(artmeans, labels)

/Users/johannaanderson/anaconda/lib/python3.5/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


0.19136070382925882